In [1]:
# console and directory access
import os
import shutil

# interacting with Amazon AWS
import boto3
from sagemaker.session import Session

# data reading and exporting  
import pandas as pd
import json
import tempfile

# parsing SEC website for data  
import requests
import time 
from bs4 import BeautifulSoup

In [2]:
file_type = 'X-17A-5'       # files looking to extract
prior2date = '20201231'     # format YYYY/MM/DD - select data prior to this data

In [3]:
bucket = "ran-s3-systemic-risk"
data_folder ="X17A5/"

# Amazon Textract client and Sagemaker session
s3 = boto3.client('s3')
session = Session()

In [4]:
os.getcwd()

'/home/ec2-user/SageMaker'

## Dealer Data Import
Parses in dealer information with accompaning CIK code for EDGAR lookup - data is taken from broker dealer information 

In [5]:
basePath = os.getcwd()+'/Input/'

In [6]:
if os.path.isfile(basePath+'secRegisteredDealers.txt'):
    
    # exporting RegisteredDealer information, loading in JSON dictionary 
    with open(basePath+'secRegisteredDealers.txt') as file:
        cik2brokers = json.load(file)
    
    # unpacking the dictionary values (all broker dealer company names)
    bdNames = [*cik2brokers]
    
else:
    print('File not found, retrieving information')
    
    # reading in the broker dealer information
    brokerDealers = pd.read_excel(basePath+'broker-dealer-data_FY2018_Sample.xlsx')

    # replace weird name formats from broker-dealer info
    params = {
            'A.G.P. / ALLIANCE GLOBAL PARTNERS CORP.':'A.G.P.',
            'ACA/PRUDENT INVESTORS PLANNING CORPORATION':'ACA',
            'BROWN, LISLE/CUMMINGS, INC.':'CUMMINGS, INC.',
            'COLLINS/BAY ISLAND SECURITIES, LLC':'COLLINS',
            'FRANKLIN/TEMPLETON DISTRIBUTORS, INC.':'FRANKLIN',
            'HCFP/CAPITAL MARKETS LLC':'CAPITAL MARKETS LLC',
            'TEMPLETON/FRANKLIN INVESTMENT SERVICES,INC.':'TEMPLETON',
            }
    brokerDealers['Company'] = brokerDealers[['Company']].replace(params)
    
    # broker dealers assigned by unique CIK Codes, casting to integer type
    bdNames = brokerDealers['CIK Code'].unique()[:-1]
    bdNames = bdNames.astype(int)
    
    # used to reference the company name from the CIK code
    cik2brokers = brokerDealers[['CIK Code', 'Company']].set_index('CIK Code')
    cik2brokers = cik2brokers.iloc[:-3].to_dict()['Company']
    
    # exporting RegisteredDealer information
    with open(basePath+'secRegisteredDealers.txt', 'w') as file:
        json.dump(cik2brokers, file)
        file.close()

In [7]:
pd.DataFrame.from_dict(cik2brokers, orient='index', columns=['Broker Dealer']).head()

,Broker Dealer
782124.0,J.P. MORGAN SECURITIES LLC
42352.0,GOLDMAN SACHS & CO. LLC
68136.0,MORGAN STANLEY & CO. LLC
91154.0,CITIGROUP GLOBAL MARKETS INC.
72267.0,"NOMURA SECURITIES INTERNATIONAL, INC."


## PDF File Extraction
Extract URL links per company filing to download accompaning X-17A-5 files from SEC EDGAR site

In [8]:
n = len(bdNames)                # number of broker/dealers
secURLS = [0]*n                 # initialize memory 
startTime = time.time()         # set tuner to track code 
completeYear = None             # initialize year to check for updates

secFormat = 'https://www.sec.gov/cgi-bin/browse-edgar?'     # SEC base url
dataSelect = 'action=getcompany&CIK={}&type={}&dateb={}'    # select params.
    
# iterate through the CIK's from the broker dealers to build lookup URLs
for name in range(n):
    companyCIK = bdNames[name][:-2]         # e.g. 782124.0 -> 782124
    url = secFormat + dataSelect.format(companyCIK, file_type, prior2date)
    secURLS[name] = url                     # storing the search URLs

In [9]:
secURLS[:5]

['https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=782124&type=X-17A-5&dateb=20201231',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=42352&type=X-17A-5&dateb=20201231',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=68136&type=X-17A-5&dateb=20201231',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=91154&type=X-17A-5&dateb=20201231',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=72267&type=X-17A-5&dateb=20201231']

In [10]:
def secParse(completeYear:str, nLinks:int, secURLS:list, cik2brokers:dict, bdNames:list, file_type:str, 
             bucket:str, subFolder:str=None):
    """
    Parses SEC website for X-17A-5 files (void function)
    :param nLinks: (int) the number of links to extract from
    :return: None
    """
    # initialize time for process to run 
    startTime = time.time()
    
    # discover all of the pdfs that you want to parse
    s3_path = session.list_s3_files(bucket, subFolder)
    
    # the URL links for each SEC company
    for url_link in range(nLinks):

        # company name for broker dealer being downloaded
        companyName = cik2brokers[bdNames[url_link]]

        # logging info for when files are being downloaded
        print('{} - Downloading {} files for {}'.format(url_link, file_type, companyName))

        # requesting HTML data link from the EDGAR search results 
        response = requests.get(secURLS[url_link], allow_redirects=True)

        # parse the HTML doc string from the response object
        s1Table = BeautifulSoup(response.text, 'html.parser') 

        # parse the html-doc string for all instance of <a href=>
        for link in s1Table.find_all('a'):
            documentURL = link.get('href')           # document links for filings

            try:
                # Check for Archives header as those are contained in the filings
                check = documentURL.split('/')[1]    
                if check == 'Archives':                 

                    # document URL link for each SEC filing for given year 
                    pdf_url = 'https://www.sec.gov' + documentURL
                    year = documentURL.split('-')[1] 
                    
                    # if years match this implies a file has been amended 
                    # data is organized linearly, by most recent issue first
                    if completeYear != year:

                        # requesting data from document links storing the files
                        pdf_storage = requests.get(pdf_url, allow_redirects=True)

                        # table from filing detail Edgar table 
                        s2Table = BeautifulSoup(pdf_storage.text, 'html.parser') 

                        # extracts all link within the filing table, filtering for pdfs
                        filing_detail = s2Table.find_all('a')
                        extract_link = [file.get('href') for file in filing_detail]

                        # filter for all pdf links from the extracted file links  
                        pdf_files = [string for string in extract_link if 'pdf' in string]
                        pdf_file = 'https://www.sec.gov' + pdf_files[-1] 

                        # storing pdf details within the folder
                        pdf_name = 'X17A5/' + cik2brokers[bdNames[url_link]] + year + '.pdf'

                        # if pdf file is stored locally avoid running script further
                        # simply continue to other company (we assume all present)
                        if pdf_name in s3_path: 
                            print('\tAll files for {} are downloaded'.format(companyName))
                            break

                        else:
                            # request the specific pdf file from the the SEC
                            pdf_storage = requests.get(pdf_file, allow_redirects=True)
                            
                            # filename for the pdf to be stored in s3
                            fileName = cik2brokers[bdNames[url_link]] + year + '.pdf'
                            
                            # save PDF contents to local file location 
                            open(fileName, 'wb').write(pdf_storage.content)
                            
                            # save contents to AWS S3 bucket
                            with open(fileName, 'rb') as data:
                                s3.upload_fileobj(data, bucket, subFolder+fileName)
                                
                            # remove local file after it has been created
                            os.remove(fileName)
                            
                            print('\tSaved {} files for {} year {}'.format(file_type, companyName, year))
                            completeYear = year 

            # if documentURL has no split greater than length of 1, false link       
            except IndexError:
                pass

    print('Time taken in seconds is {}'.format(time.time() - startTime))

In [ ]:
# call function to parse data from the SEC -> port to s3
secParse(prior2date[:4], n, secURLS, cik2brokers, bdNames, file_type, bucket, data_folder)

0 - Downloading X-17A-5 files for J.P. MORGAN SECURITIES LLC
	All files for J.P. MORGAN SECURITIES LLC are downloaded
1 - Downloading X-17A-5 files for GOLDMAN SACHS & CO. LLC
	All files for GOLDMAN SACHS & CO. LLC are downloaded
2 - Downloading X-17A-5 files for MORGAN STANLEY & CO. LLC
	All files for MORGAN STANLEY & CO. LLC are downloaded
3 - Downloading X-17A-5 files for CITIGROUP GLOBAL MARKETS INC.
	All files for CITIGROUP GLOBAL MARKETS INC. are downloaded
4 - Downloading X-17A-5 files for NOMURA SECURITIES INTERNATIONAL, INC.
	All files for NOMURA SECURITIES INTERNATIONAL, INC. are downloaded
5 - Downloading X-17A-5 files for WELLS FARGO SECURITIES, LLC
	All files for WELLS FARGO SECURITIES, LLC are downloaded
6 - Downloading X-17A-5 files for BARCLAYS CAPITAL INC.
	All files for BARCLAYS CAPITAL INC. are downloaded
7 - Downloading X-17A-5 files for HSBC SECURITIES (USA) INC.
	All files for HSBC SECURITIES (USA) INC. are downloaded
8 - Downloading X-17A-5 files for DEUTSCHE BAN

	All files for WALLEYE TRADING LLC are downloaded
70 - Downloading X-17A-5 files for BNY MELLON CAPITAL MARKETS, LLC
	All files for BNY MELLON CAPITAL MARKETS, LLC are downloaded
71 - Downloading X-17A-5 files for STIFEL, NICOLAUS & COMPANY, INCORPORATED
	All files for STIFEL, NICOLAUS & COMPANY, INCORPORATED are downloaded
72 - Downloading X-17A-5 files for AMERICAN ENTERPRISE INVESTMENT SERVICES INC.
	All files for AMERICAN ENTERPRISE INVESTMENT SERVICES INC. are downloaded
73 - Downloading X-17A-5 files for JANNEY MONTGOMERY SCOTT LLC
	All files for JANNEY MONTGOMERY SCOTT LLC are downloaded
74 - Downloading X-17A-5 files for APEX CLEARING CORPORATION
	All files for APEX CLEARING CORPORATION are downloaded
75 - Downloading X-17A-5 files for FIDELITY BROKERAGE SERVICES LLC
	All files for FIDELITY BROKERAGE SERVICES LLC are downloaded
76 - Downloading X-17A-5 files for NATIONAL BANK OF CANADA FINANCIAL INC.
	All files for NATIONAL BANK OF CANADA FINANCIAL INC. are downloaded
77 - Down

	All files for HARDCASTLE TRADING USA L.L.C. are downloaded
139 - Downloading X-17A-5 files for GTS SECURITIES LLC
	All files for GTS SECURITIES LLC are downloaded
140 - Downloading X-17A-5 files for EVERCORE GROUP L.L.C.
	All files for EVERCORE GROUP L.L.C. are downloaded
141 - Downloading X-17A-5 files for FIFTH THIRD SECURITIES, INC.
	All files for FIFTH THIRD SECURITIES, INC. are downloaded
142 - Downloading X-17A-5 files for UBS FINANCIAL SERVICES INCORPORATED OF PUERTO RICO
	All files for UBS FINANCIAL SERVICES INCORPORATED OF PUERTO RICO are downloaded
143 - Downloading X-17A-5 files for MESIROW FINANCIAL, INC.
	All files for MESIROW FINANCIAL, INC. are downloaded
144 - Downloading X-17A-5 files for HOULIHAN LOKEY CAPITAL, INC.
	All files for HOULIHAN LOKEY CAPITAL, INC. are downloaded
145 - Downloading X-17A-5 files for INSTINET, LLC
	All files for INSTINET, LLC are downloaded
146 - Downloading X-17A-5 files for STOCKCROSS FINANCIAL SERVICES, INC.
	All files for STOCKCROSS FINA

	All files for NATHAN HALE CAPITAL, LLC are downloaded
208 - Downloading X-17A-5 files for LEK SECURITIES CORPORATION
	All files for LEK SECURITIES CORPORATION are downloaded
209 - Downloading X-17A-5 files for PARK HILL GROUP LLC
	All files for PARK HILL GROUP LLC are downloaded
210 - Downloading X-17A-5 files for ELECTRONIC TRANSACTION CLEARING, INC.
	All files for ELECTRONIC TRANSACTION CLEARING, INC. are downloaded
211 - Downloading X-17A-5 files for BNY MELLON SECURITIES CORPORATION
	All files for BNY MELLON SECURITIES CORPORATION are downloaded
212 - Downloading X-17A-5 files for KKR CAPITAL MARKETS LLC
	All files for KKR CAPITAL MARKETS LLC are downloaded
213 - Downloading X-17A-5 files for PNC INVESTMENTS LLC
	All files for PNC INVESTMENTS LLC are downloaded
214 - Downloading X-17A-5 files for BERENBERG CAPITAL MARKETS LLC
	All files for BERENBERG CAPITAL MARKETS LLC are downloaded
215 - Downloading X-17A-5 files for DASH FINANCIAL TECHNOLOGIES LLC
	All files for DASH FINANCIAL

	All files for BOK FINANCIAL SECURITIES, INC. are downloaded
274 - Downloading X-17A-5 files for PFS INVESTMENTS INC.
	All files for PFS INVESTMENTS INC. are downloaded
275 - Downloading X-17A-5 files for NORTHERN TRUST SECURITIES, INC.
	All files for NORTHERN TRUST SECURITIES, INC. are downloaded
276 - Downloading X-17A-5 files for NABSECURITIES, LLC
	All files for NABSECURITIES, LLC are downloaded
277 - Downloading X-17A-5 files for PRUDENTIAL INVESTMENT MANAGEMENT SERVICES LLC
	All files for PRUDENTIAL INVESTMENT MANAGEMENT SERVICES LLC are downloaded
278 - Downloading X-17A-5 files for BLOOMBERG TRADEBOOK LLC
	All files for BLOOMBERG TRADEBOOK LLC are downloaded
279 - Downloading X-17A-5 files for FIRST TRUST PORTFOLIOS L.P.
	All files for FIRST TRUST PORTFOLIOS L.P. are downloaded
280 - Downloading X-17A-5 files for T. ROWE PRICE INVESTMENT SERVICES, INC.
	All files for T. ROWE PRICE INVESTMENT SERVICES, INC. are downloaded
281 - Downloading X-17A-5 files for CREWS & ASSOCIATES, I

...

	Saved X-17A-5 files for THE HINA GROUP, INC. year 14
	Saved X-17A-5 files for THE HINA GROUP, INC. year 13
	Saved X-17A-5 files for THE HINA GROUP, INC. year 12
	Saved X-17A-5 files for THE HINA GROUP, INC. year 11
	Saved X-17A-5 files for THE HINA GROUP, INC. year 10
	Saved X-17A-5 files for THE HINA GROUP, INC. year 09
	Saved X-17A-5 files for THE HINA GROUP, INC. year 08
	Saved X-17A-5 files for THE HINA GROUP, INC. year 07
	Saved X-17A-5 files for THE HINA GROUP, INC. year 06
	Saved X-17A-5 files for THE HINA GROUP, INC. year 05
	Saved X-17A-5 files for THE HINA GROUP, INC. year 04
1838 - Downloading X-17A-5 files for CAPITAL MANAGEMENT OF THE CAROLINAS, LLC
	Saved X-17A-5 files for CAPITAL MANAGEMENT OF THE CAROLINAS, LLC year 20
	Saved X-17A-5 files for CAPITAL MANAGEMENT OF THE CAROLINAS, LLC year 19
	Saved X-17A-5 files for CAPITAL MANAGEMENT OF THE CAROLINAS, LLC year 18
	Saved X-17A-5 files for CAPITAL MANAGEMENT OF THE CAROLINAS, LLC year 17
	Saved X-17A-5 files for CAPITAL

	Saved X-17A-5 files for STRAUS CAPITAL, LLC year 14
	Saved X-17A-5 files for STRAUS CAPITAL, LLC year 13
	Saved X-17A-5 files for STRAUS CAPITAL, LLC year 12
	Saved X-17A-5 files for STRAUS CAPITAL, LLC year 11
	Saved X-17A-5 files for STRAUS CAPITAL, LLC year 10
	Saved X-17A-5 files for STRAUS CAPITAL, LLC year 09
	Saved X-17A-5 files for STRAUS CAPITAL, LLC year 08
	Saved X-17A-5 files for STRAUS CAPITAL, LLC year 07
1846 - Downloading X-17A-5 files for TITLEIST ASSET MANAGEMENT, LTD.
	Saved X-17A-5 files for TITLEIST ASSET MANAGEMENT, LTD. year 20
	Saved X-17A-5 files for TITLEIST ASSET MANAGEMENT, LTD. year 19
	Saved X-17A-5 files for TITLEIST ASSET MANAGEMENT, LTD. year 18
	Saved X-17A-5 files for TITLEIST ASSET MANAGEMENT, LTD. year 17
	Saved X-17A-5 files for TITLEIST ASSET MANAGEMENT, LTD. year 16
	Saved X-17A-5 files for TITLEIST ASSET MANAGEMENT, LTD. year 15
	Saved X-17A-5 files for TITLEIST ASSET MANAGEMENT, LTD. year 14
	Saved X-17A-5 files for TITLEIST ASSET MANAGEMENT, 

	Saved X-17A-5 files for MORGAN CREEK CAPITAL DISTRIBUTORS, LLC year 20
	Saved X-17A-5 files for MORGAN CREEK CAPITAL DISTRIBUTORS, LLC year 19
	Saved X-17A-5 files for MORGAN CREEK CAPITAL DISTRIBUTORS, LLC year 18
	Saved X-17A-5 files for MORGAN CREEK CAPITAL DISTRIBUTORS, LLC year 17
	Saved X-17A-5 files for MORGAN CREEK CAPITAL DISTRIBUTORS, LLC year 16
	Saved X-17A-5 files for MORGAN CREEK CAPITAL DISTRIBUTORS, LLC year 15
	Saved X-17A-5 files for MORGAN CREEK CAPITAL DISTRIBUTORS, LLC year 14
	Saved X-17A-5 files for MORGAN CREEK CAPITAL DISTRIBUTORS, LLC year 13
	Saved X-17A-5 files for MORGAN CREEK CAPITAL DISTRIBUTORS, LLC year 12
1858 - Downloading X-17A-5 files for FMN CAPITAL CORPORATION
	Saved X-17A-5 files for FMN CAPITAL CORPORATION year 20
	Saved X-17A-5 files for FMN CAPITAL CORPORATION year 19
	Saved X-17A-5 files for FMN CAPITAL CORPORATION year 18
	Saved X-17A-5 files for FMN CAPITAL CORPORATION year 17
	Saved X-17A-5 files for FMN CAPITAL CORPORATION year 16
	Saved

	Saved X-17A-5 files for AUTOMATED EQUITY FINANCE MARKETS, INC. year 09
1866 - Downloading X-17A-5 files for QUANTEDGE CAPITAL USA INC.
	Saved X-17A-5 files for QUANTEDGE CAPITAL USA INC. year 20
	Saved X-17A-5 files for QUANTEDGE CAPITAL USA INC. year 19
	Saved X-17A-5 files for QUANTEDGE CAPITAL USA INC. year 18
	Saved X-17A-5 files for QUANTEDGE CAPITAL USA INC. year 17
1867 - Downloading X-17A-5 files for HILCO CORPORATE FINANCE, LLC
	Saved X-17A-5 files for HILCO CORPORATE FINANCE, LLC year 20
	Saved X-17A-5 files for HILCO CORPORATE FINANCE, LLC year 19
	Saved X-17A-5 files for HILCO CORPORATE FINANCE, LLC year 18
	Saved X-17A-5 files for HILCO CORPORATE FINANCE, LLC year 17
1868 - Downloading X-17A-5 files for US ARMA PARTNERS LP
	Saved X-17A-5 files for US ARMA PARTNERS LP year 20
	Saved X-17A-5 files for US ARMA PARTNERS LP year 19
	Saved X-17A-5 files for US ARMA PARTNERS LP year 18
	Saved X-17A-5 files for US ARMA PARTNERS LP year 17
	Saved X-17A-5 files for US ARMA PARTNERS

	Saved X-17A-5 files for LEWIS YOUNG ROBERTSON & BURNINGHAM, INC. year 02
1877 - Downloading X-17A-5 files for BERCHWOOD PARTNERS LLC
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 20
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 19
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 18
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 17
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 16
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 15
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 14
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 13
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 12
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 11
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 10
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 09
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 08
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 07
	Saved X-17A-5 files for BERCHWOOD PARTNERS LLC year 06
	Saved X-17A-5 files for B